# 智能体Agent控制机器人

同济子豪兄 2024-9-7

## 录音+语音识别

In [4]:
!ssh pi@192.168.31.146 "python TonyPi/OpenVINO/utils_asr.py"

寮�濮� 6 绉掑綍闊�

Recording WAVE '/home/pi/TonyPi/OpenVINO/temp/speech_record.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Mono



褰曢煶缁撴潫
寮�濮嬭��闊宠瘑鍒�
璇�闊宠瘑鍒�缁撴灉锛� 鍏堟嫎涓�韬�鍐嶆尌鎸ユ墜锛屽線鍚庨��涓ゆ�ワ紝鍐嶅線鍓嶈蛋涓夋�ワ紝鐒跺悗鍋氫釜搴嗙�濈殑鍔ㄣ��
璇�闊宠瘑鍒�缁撴灉宸插啓鍏�txt鏂囦欢


语音识别结果已写入`~/TonyPi/OpenVINO/temp/speech_recognition.txt`文件

## 读取语音识别结果

In [5]:
!scp pi@192.168.31.146:~/TonyPi/OpenVINO/temp/speech_recognition.txt temp/speech_recognition.txt 

In [7]:
with open('temp/speech_recognition.txt', 'r', encoding='utf-8') as f:
    speech_result = f.read()

In [8]:
speech_result

'先拘个躬再挥挥手，往后退两步，再往前走三步，然后做个庆祝的动。'

## 导入工具包

In [9]:
import openvino_genai as ov_genai

## 载入IR格式的大模型

In [10]:
pipe = ov_genai.LLMPipeline("Qwen2-7B-Instruct-int4-ov", "GPU")

## 函数：智能体Agent编排动作

In [11]:
robot_order_template = '''
你是我的机器人，请你根据我的指令，以列表形式输出接下来要运行的对应函数和你给我的回复
你只需要回答一个列表即可，不要回答任何中文
【以下是所有动作函数】
站立：stand()
前进一步：move_forward()
后退一步：move_back()
向左平移移动一步：move_left()
向右平移移动一步：move_right()
向左旋转移动：turn_left()
向右旋转移动：turn_right()
鞠躬：bow()
挥手打招呼：wave()
扭腰：twist()
捶胸庆祝：celebrate()
下蹲：squat()
踢右脚：right_shot()
踢左脚：left_shot()
仰卧起坐：sit_ups()
巡线模式：athletics()
播放音乐并跳舞：rap()

【以下是一些具体的例子】
我的指令：请你先鞠个躬，然后挥挥手。你回复：['bow()', 'wave()']
我的指令：先前进，再后退，向左转一点，再向右平移。你回复：['move_forward()', 'move_back()', 'turn_left()', 'move_right()']
我的指令：先蹲下，再站起来，最后做个庆祝的动作。你回复：['squat()', 'stand()', 'celebrate()']
我的指令：向前走两步，向后退三步。你回复：['move_forward()', 'move_forward()', 'move_back()', 'move_back()', 'move_back()']

【我现在的指令是】
{}
'''

In [12]:
def agent_action_plan(question="先鞠个躬，再打个招呼，蹲下，最后站起来"):
    prompt_human = robot_order_template.format(question)
    prompt_machine = "<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n<|im_start|>assistant\n".format(prompt_human)
    result = pipe.generate(prompt_machine)
    agent_plan_str = result.texts[0]

    # 写入txt文件
    agent_plan_txt = 'temp/agent_plan.txt'
    with open(agent_plan_txt, 'w') as f:
        f.write(agent_plan_str)
        # print('动作编排已写入txt文件')

    agent_plan_list = eval(agent_plan_str)
    return agent_plan_list

## 测试

In [17]:
agent_plan_list = agent_action_plan()

In [18]:
agent_plan_list

['bow()', 'wave()', 'squat()', 'stand()']

In [19]:
agent_plan_list = agent_action_plan('请你先向前两步，再向后退一步，向左微微转动，最后做个健身的动作')

In [20]:
agent_plan_list

['move_forward()', 'move_forward()', 'move_back()', 'turn_left()', 'sit_ups()']

In [8]:
agent_plan_list = agent_action_plan('先鞠个躬，然后打个招呼，向前走三步，再向后退两步，向左微微转动，最后做个庆祝的动作')

In [9]:
agent_plan_list

['bow()',
 'wave()',
 'move_forward()',
 'move_forward()',
 'move_forward()',
 'move_back()',
 'move_back()',
 'turn_left()',
 'celebrate()']

## 传入语音识别结果，编排生成动作列表

In [13]:
agent_plan_list = agent_action_plan(speech_result)

In [14]:
agent_plan_list

['bow()',
 'wave()',
 'move_back()',
 'move_back()',
 'move_forward()',
 'move_forward()',
 'move_forward()',
 'celebrate()']

## 把动作编排txt文件传到开发板

In [15]:
!scp temp/agent_plan.txt pi@192.168.31.146:~/TonyPi/OpenVINO/temp/

## AIPC远程触发机器人运动

In [1]:
!ssh pi@192.168.31.146 "python ~/TonyPi/OpenVINO/utils_robot.py"

寮�濮嬩緷娆℃墽琛屽姩浣�
杞藉叆鍔ㄤ綔鍒楄〃 ['bow()', 'wave()', 'move_back()', 'move_back()', 'move_forward()', 'move_forward()', 'move_forward()', 'celebrate()']
寮�濮嬪姩浣� bow()
寮�濮嬪姩浣� wave()
寮�濮嬪姩浣� move_back()
寮�濮嬪姩浣� move_back()
寮�濮嬪姩浣� move_forward()
寮�濮嬪姩浣� move_forward()
寮�濮嬪姩浣� move_forward()
寮�濮嬪姩浣� celebrate()
